In [1]:
from keras.datasets import cifar10
import tensorflow as tf
from tensorflow import keras
import numpy as np
import math
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical
from keras import layers
from keras import models
from keras import optimizers
from time import time as t
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
# loading CIFAR10 data
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# normalizing data
train_images = train_images.reshape((50000, 32, 32, 3)) / 255.0
test_images = test_images.reshape((10000, 32, 32, 3)) / 255.0

In [3]:
# m is the number of training examples
m = train_images.shape[0]
shuffled_indices = np.random.permutation(m)
# shuffling the training data randomely
train_images_shuffled = train_images[shuffled_indices]
train_labels_shuffled = train_labels[shuffled_indices]

# setting 0.20 of the training data to validation set randomly


# one hot encoding of the labels
train_labels_shuffled = to_categorical(train_labels_shuffled, num_classes=10)
test_labels = to_categorical(test_labels, num_classes=10)

## K - Fold Validation

In [8]:
# training function for each fold using data augmentation 
def fold(model, training_set, training_labels, validation_set, validation_set_labels):
    train_datagen = ImageDataGenerator(
        width_shift_range=0.2,  # randomly shift images horizontally 
        height_shift_range=0.2,  # randomly shift images vertically 
        shear_range=0.2,        # randomly shear images
        horizontal_flip=True)  # flip images horizontally

    validation_datagen = ImageDataGenerator()

    train_generator = train_datagen.flow(training_set, training_labels, batch_size=32)
    validation_generator = validation_datagen.flow(validation_set, validation_set_labels, batch_size=32)
    history_dataaug_model5 = model.fit_generator(train_generator, validation_data=validation_generator,
                                                 validation_steps=len(training_set) / 32,
                                                 steps_per_epoch=len(training_set) / 32,
                                                 epochs=30, verbose=2)

    score = model.evaluate(validation_set, validation_set_labels, batch_size=128, verbose=0)
    return score[1]

In [9]:
# Using the winner architecture (arch 5) for training the data in each fold and storing the score 
def k_fold_validation(training_set, training_labels, n):
    # scores stores the accuracy of 5 folds
    scores = []

   
    fold_size = math.floor(len(training_set) / n)
    for i in range(n):
        print("---------- Fold", i+1, "----------")
        fold_validation_set = training_set[i * fold_size:i * fold_size + fold_size]
        fold_validation_set_labels = training_labels[i * fold_size:i * fold_size + fold_size]
        fold_training_set = np.delete(training_set, slice(i * fold_size, i * fold_size + fold_size), axis=0)
        fold_training_labels = np.delete(training_labels, slice(i * fold_size, i * fold_size + fold_size), axis=0)
        model_5 = models.Sequential()
        model_5.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
        model_5.add(layers.Conv2D(64, (3, 3), activation='relu'))
        model_5.add(layers.Conv2D(64, (3, 3), activation='relu', strides=2))
        model_5.add(layers.Conv2D(64, (3, 3), activation='relu'))
        model_5.add(layers.Conv2D(128, (3, 3), activation='relu', strides=2))
        model_5.add(layers.Conv2D(128, (3, 3), activation='relu'))
        model_5.add(layers.Flatten())
        model_5.add(layers.Dropout(0.5))
        model_5.add(layers.Dense(64, activation='relu'))
        model_5.add(layers.normalization.BatchNormalization())
        model_5.add(layers.Dense(10, activation='softmax'))

        model_5.summary()

        model_5.compile(optimizer='rmsprop',
                        loss='categorical_crossentropy',
                        metrics=['accuracy'])
        # score of each fold
        s = fold(model_5, fold_training_set, fold_training_labels, fold_validation_set, fold_validation_set_labels)

        scores.append(s)

    print(scores)
    
    # computes the average of 5 accuracies for 5 folds
    scores_average = np.average(scores)
    print("Average score =", scores_average)


k_fold_validation(train_images_shuffled, train_labels_shuffled, 5)


---------- Fold 1 ----------
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 28, 28, 64)        18496     
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 13, 13, 64)        36928     
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 11, 11, 64)        36928     
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 5, 5, 128)         73856     
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 3, 3, 128)         147584    
_________________________________________________________________
flatten_4 (Flatten)          (None, 1152)      

Epoch 25/30
 - 13s - loss: 0.7245 - acc: 0.7521 - val_loss: 0.6981 - val_acc: 0.7617
Epoch 26/30
 - 13s - loss: 0.7215 - acc: 0.7508 - val_loss: 0.6297 - val_acc: 0.7850
Epoch 27/30
 - 13s - loss: 0.7144 - acc: 0.7541 - val_loss: 0.8418 - val_acc: 0.7350
Epoch 28/30
 - 13s - loss: 0.7130 - acc: 0.7534 - val_loss: 0.6090 - val_acc: 0.7932
Epoch 29/30
 - 13s - loss: 0.7005 - acc: 0.7599 - val_loss: 0.5663 - val_acc: 0.8090
Epoch 30/30
 - 13s - loss: 0.6919 - acc: 0.7652 - val_loss: 0.7083 - val_acc: 0.7668
---------- Fold 3 ----------
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_31 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 28, 28, 64)        18496     
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 13, 13, 64)        36928     

Epoch 19/30
 - 13s - loss: 0.7653 - acc: 0.7371 - val_loss: 0.6521 - val_acc: 0.7827
Epoch 20/30
 - 13s - loss: 0.7616 - acc: 0.7381 - val_loss: 0.7544 - val_acc: 0.7465
Epoch 21/30
 - 13s - loss: 0.7495 - acc: 0.7436 - val_loss: 0.6611 - val_acc: 0.7698
Epoch 22/30
 - 13s - loss: 0.7366 - acc: 0.7514 - val_loss: 0.8066 - val_acc: 0.7296
Epoch 23/30
 - 13s - loss: 0.7370 - acc: 0.7483 - val_loss: 0.7152 - val_acc: 0.7584
Epoch 24/30
 - 13s - loss: 0.7197 - acc: 0.7545 - val_loss: 0.6801 - val_acc: 0.7771
Epoch 25/30
 - 13s - loss: 0.7163 - acc: 0.7556 - val_loss: 0.7269 - val_acc: 0.7547
Epoch 26/30
 - 13s - loss: 0.7101 - acc: 0.7565 - val_loss: 0.7185 - val_acc: 0.7590
Epoch 27/30
 - 13s - loss: 0.6996 - acc: 0.7630 - val_loss: 0.5862 - val_acc: 0.7949
Epoch 28/30
 - 13s - loss: 0.6895 - acc: 0.7632 - val_loss: 0.6626 - val_acc: 0.7842
Epoch 29/30
 - 13s - loss: 0.6911 - acc: 0.7626 - val_loss: 0.7885 - val_acc: 0.7583
Epoch 30/30
 - 13s - loss: 0.6796 - acc: 0.7695 - val_loss: 0.601

In [ ]:
# As it is shown in the result, the average of the accuracies for 5-fold is 78%. In simple hold-out validation 
# we also got 78% accuracy which is similar to the k-fold result. If we increase the number of epochs, 
# their difference may be increased. 
